In [1]:
import scipy.io as sio
import tensorflow as tf
import numpy as np

from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Dense,
    ConvLSTM2D,
    GlobalMaxPooling2D, 
    )

In [2]:
mat = sio.loadmat('../../data/processed/sevenThirty/s05.mat')
mi = mat['mi']
real = mat['real']
class_mi = mat['class_mi']
class_real = mat['class_real']

In [3]:
mi_reshape = np.empty([mi.shape[2], 64, 1024])

mi_intermediate = np.empty([mi.shape[2], 16, 64, 64])

for i in range(0, mi.shape[2]):
    mi_reshape[i, :, :] = mi[:, :, i]
    
    for n in range(0, mi_intermediate.shape[1]):

        startIDX = (64 * n)
        endIDX = startIDX + 64
        
        mi_intermediate[i, n, :, :] = mi_reshape[i, :, startIDX:endIDX]
    
mi_tensor = tf.reshape(mi_intermediate, shape = (mi.shape[2], mi_intermediate.shape[1], 64, 64, 1))

class_mi_tensor = tf.reshape(class_mi , mi.shape[2])

class_mi_tensor = class_mi_tensor - 1

In [4]:
real_reshape = np.empty([real.shape[2], 64, 1024])

real_intermediate = np.empty([real.shape[2], 16, 64, 64])

for i in range(0, real.shape[2]):
    real_reshape[i, :, :] = real[:, :, i]
    
    for n in range(0, real_intermediate.shape[1]):

        startIDX = (64 * n)
        endIDX = startIDX + 64
        
        real_intermediate[i, n, :, :] = real_reshape[i, :, startIDX:endIDX]
    
real_tensor = tf.reshape(real_intermediate, shape = (real.shape[2], real_intermediate.shape[1], 64, 64, 1))

class_real_tensor = tf.reshape(class_real , real.shape[2])

class_real_tensor = class_real_tensor - 1

In [5]:
opt = tf.keras.optimizers.RMSprop(
    lr=0.001)

loss = tf.keras.losses.BinaryCrossentropy()

In [17]:
model = Sequential()
model.add(ConvLSTM2D(
    filters=64,
    kernel_size=(4, 4),
#     padding='same',
#     data_format='channels_last',
#     return_sequences=False,
    input_shape=(16, 64, 64, 1)
    ))
model.add(GlobalMaxPooling2D())
# model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [12]:
model.compile(
    optimizer=opt,
    loss=loss,
    metrics=['accuracy'])

In [18]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_4 (ConvLSTM2D)  (None, 61, 61, 64)        266496    
_________________________________________________________________
global_max_pooling2d_2 (Glob (None, 64)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 256)               16640     
_________________________________________________________________
dense_12 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_13 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 65        
Total params: 324,353
Trainable params: 324,353
Non-trainable params: 0
________________________________________________

In [8]:
model.fit(
    x=mi_tensor,
    y=class_mi_tensor,
    batch_size=mi.shape[2],
    validation_data=(real_tensor, class_real_tensor),
    epochs=10,
    verbose=2)

Epoch 1/10
1/1 - 25s - loss: 0.7026 - accuracy: 0.5000 - val_loss: 0.7027 - val_accuracy: 0.5000
Epoch 2/10
1/1 - 18s - loss: 0.7010 - accuracy: 0.5000 - val_loss: 0.6942 - val_accuracy: 0.5000
Epoch 3/10
1/1 - 19s - loss: 0.6939 - accuracy: 0.5000 - val_loss: 0.6950 - val_accuracy: 0.5000
Epoch 4/10
1/1 - 19s - loss: 0.6938 - accuracy: 0.5000 - val_loss: 0.6939 - val_accuracy: 0.5000
Epoch 5/10
1/1 - 20s - loss: 0.6934 - accuracy: 0.5000 - val_loss: 0.6943 - val_accuracy: 0.5000
Epoch 6/10
1/1 - 18s - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6940 - val_accuracy: 0.5000
Epoch 7/10
1/1 - 18s - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6943 - val_accuracy: 0.5000
Epoch 8/10
1/1 - 18s - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6942 - val_accuracy: 0.5000
Epoch 9/10
1/1 - 19s - loss: 0.6930 - accuracy: 0.5050 - val_loss: 0.6945 - val_accuracy: 0.5000
Epoch 10/10
1/1 - 19s - loss: 0.6930 - accuracy: 0.5000 - val_loss: 0.6944 - val_accuracy: 0.5000


In [6]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits=10, shuffle=True)

In [20]:
mi_tensor.shape

TensorShape([200, 16, 64, 64, 1])

In [ ]:
for train, test in kfold.split(mi_tensor, class_mi_tensor):
    
    model = Sequential()
    model.add(ConvLSTM2D(
        filters=4,
        kernel_size=(4, 4),
    #     padding='same',
    #     data_format='channels_last',
    #     return_sequences=False,
    #     input_shape=(64, 1024, 1)
        ))
    model.add(GlobalMaxPooling2D())
    # model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(
    optimizer=opt,
    loss=loss,
    metrics=['accuracy'])
    
    model.fit(
    x=tf.gather(mi_tensor, train),
    y=tf.gather(class_mi_tensor, train),
    batch_size=180,
    validation_data=(tf.gather(mi_tensor, test), tf.gather(class_mi_tensor, test)),
    epochs=10,
    verbose=2)
#     print(train.shape)
#     print(test.shape)
# #     print(class_mi_tensor[])
#     print(tf.gather(mi_tensor, test).shape)

Epoch 1/10
1/1 - 23s - loss: 0.6946 - accuracy: 0.5000 - val_loss: 0.6934 - val_accuracy: 0.5000
Epoch 2/10
1/1 - 18s - loss: 0.6935 - accuracy: 0.5000 - val_loss: 0.6948 - val_accuracy: 0.5000
Epoch 3/10
1/1 - 18s - loss: 0.6944 - accuracy: 0.5000 - val_loss: 0.6928 - val_accuracy: 0.5000
Epoch 4/10
1/1 - 18s - loss: 0.6930 - accuracy: 0.5000 - val_loss: 0.6928 - val_accuracy: 0.5000
Epoch 5/10
1/1 - 20s - loss: 0.6929 - accuracy: 0.5000 - val_loss: 0.6922 - val_accuracy: 0.6000
Epoch 6/10
1/1 - 22s - loss: 0.6924 - accuracy: 0.5167 - val_loss: 0.6924 - val_accuracy: 0.5000
Epoch 7/10
1/1 - 23s - loss: 0.6924 - accuracy: 0.5000 - val_loss: 0.6923 - val_accuracy: 0.5000
Epoch 8/10
1/1 - 21s - loss: 0.6928 - accuracy: 0.5000 - val_loss: 0.6936 - val_accuracy: 0.5000
Epoch 9/10
1/1 - 24s - loss: 0.6936 - accuracy: 0.5000 - val_loss: 0.6924 - val_accuracy: 0.5000
Epoch 10/10
1/1 - 26s - loss: 0.6926 - accuracy: 0.5000 - val_loss: 0.6921 - val_accuracy: 0.5000
Epoch 1/10


In [15]:
model_cnn_basic = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(128, (4,4), activation='relu', input_shape=(64, 1024, 1)),
  tf.keras.layers.MaxPooling2D(3, 3),
  tf.keras.layers.Conv2D(128, (4,4), activation='relu'),
  tf.keras.layers.MaxPooling2D(3,3),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

In [16]:
model_cnn_basic.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 61, 1021, 128)     2176      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 20, 340, 128)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 17, 337, 128)      262272    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 112, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 71680)             0         
_________________________________________________________________
dense_8 (Dense)              (None, 512)               36700672  
_________________________________________________________________
dense_9 (Dense)              (None, 128)              

In [10]:
model_cnn_basic.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [11]:
tf.reshape(new_mat, shape = (200, 64, 1024, 1))

<tf.Tensor: shape=(200, 64, 1024, 1), dtype=float64, numpy=
array([[[[-9.09148312e+00],
         [-9.18334103e+00],
         [-9.13973331e+00],
         ...,
         [-8.90516663e+00],
         [-8.84207630e+00],
         [-8.71076107e+00]],

        [[-1.06371918e+02],
         [-1.07051544e+02],
         [-1.06715981e+02],
         ...,
         [-1.05157043e+02],
         [-1.04579742e+02],
         [-1.03482391e+02]],

        [[-5.67636108e+02],
         [-5.70092834e+02],
         [-5.68882690e+02],
         ...,
         [-5.63805115e+02],
         [-5.61303833e+02],
         [-5.56863708e+02]],

        ...,

        [[ 4.56447852e+04],
         [ 8.72393359e+04],
         [ 4.82432930e+04],
         ...,
         [ 1.31842148e+04],
         [ 6.85449688e+04],
         [-1.93912305e+04]],

        [[ 3.85506484e+04],
         [ 8.26777812e+04],
         [ 4.24533789e+04],
         ...,
         [ 4.46134180e+03],
         [ 6.29898047e+04],
         [-2.59011992e+04]],

      

In [12]:
np.array(mat['class_mi'])[0] - 1

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1], dtype=uint8)

In [13]:
x = tf.reshape(new_mat, shape = (200, 64, 1024, 1))
y = tf.reshape(mat['class_mi'] , 200)

In [14]:
y

<tf.Tensor: shape=(200,), dtype=uint8, numpy=
array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2], dtype=uint8)>

In [15]:
model_cnn_basic.fit(x, y,verbose = 1)

7/7 [==============================] - 18s 2s/step - loss: 237893.3278


In [16]:
tf.reshape(mat['class_mi'], 200)

<tf.Tensor: shape=(200,), dtype=uint8, numpy=
array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2], dtype=uint8)>

In [17]:
mat['class_mi'].transpose().shape

(200, 1)